
# Process raw avi videos via the trained DLC network

In [ ]:
# imports
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import shutil
import deeplabcut as dlc
import paths
import functions_bondjango as bd
import functions_misc as fm
import yaml

# define the config_path
config_path = paths.config_path

In [ ]:
try:
    # get the target video path
    video_path = snakemake.input[0]
    out_path = snakemake.output[0]
    video_data = yaml.load(snakemake.params.file_info, Loader=yaml.FullLoader)

except NameError:
    # define the search string
    search_string = 'result:succ, lighting:normal, rig:miniscope, imaging:doric'
    # query the database for data to plot
    data_all = bd.query_database('video_experiment', search_string)
    video_data = data_all[0]
    video_path = video_data['avi_path']
    # assemble the output path
    out_path = video_path.replace('.avi', '_dlc.h5')

In [ ]:
# analyze the video
# dlc.analyze_videos?
dlc.analyze_videos(config_path, [video_path], destfolder=paths.temp_path)

In [ ]:
# filter the data
dlc.filterpredictions(config_path, [video_path],filtertype='median', 
                      windowlength=11, destfolder=paths.temp_path, save_as_csv=False)
# dlc.filterpredictions?

In [ ]:
# move and rename the file

# get a list of the files present in the temp folder
origin_file = [el for el in os.listdir(paths.temp_path) if ('filtered.h5' in el) and (video_data['slug'] in fm.slugify(el))]

assert len(origin_file) > 0, 'The target file was not found'

# rename the file to the final path
shutil.move(os.path.join(paths.temp_path, origin_file[0]), out_path)

In [ ]:
# update the bondjango entry (need to sort out some fields)
ori_data = video_data.copy()
ori_data['dlc_path'] = out_path
mouse = ori_data['mouse']
ori_data['mouse'] = '/'.join((paths.bondjango_url, 'mouse', mouse, ''))
ori_data['experiment_type'] = '/'.join((paths.bondjango_url, 'experiment_type', 'Free_behavior', ''))

update_url = '/'.join((paths.bondjango_url, 'video_experiment', ori_data['slug'], ''))
output_entry = bd.update_entry(update_url, ori_data)

print(output_entry.status_code)